Data published in [Shiraishi et al, 2024, Cancer-specific epigenome identifies oncogenic hijacking by nuclear factor I family proteins for medulloblastoma progression](https://pubmed.ncbi.nlm.nih.gov/38834071/).  
Available in [GSE243609](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE243609).  
Comparison of available trajectory methods: https://biocellgen-public.svi.edu.au/mig_2019_scrnaseq-workshop/trajectory-inference.html


In [ ]:
Sys.setenv(LANGUAGE = "en") # set language to "ja" if you prefer

library(Seurat)
library(tidyverse)
library(harmony)
library(ggplot2)
library(future)
library(mclust)
library(patchwork)
library(ggalluvial)
library(ggrepel)
library(RColorBrewer)
library(slingshot)
library(igraph)
library(ggbeeswarm)

set.seed(47);

# Data processing and clustering

In [ ]:
# Set parallel execution settings
future::plan("multisession", workers = as.integer(availableCores()/2), gc=TRUE)
options(future.globals.maxSize = 1024*8*1024^2) # Set max variable size to 8Gb

In [ ]:
# Load data functions
DATA_DIR = file.path('data','GSE243609') # change this if you put your data somewhere other than ./data/GSE243609/
DATA_LOCATIONS <- list(
    gnp = c(file.path(DATA_DIR,'GSM7791840_032103_filtered_feature_bc_matrix.h5'),
           file.path(DATA_DIR,'GSM7791839_032103_fragments.tsv.gz')),
    pnc = c(file.path(DATA_DIR,'GSM7791842_RS03056_filtered_feature_bc_matrix.h5'),
            file.path(DATA_DIR,'GSM7791841_RS03056_fragments.tsv.gz')),
    tumor = c(file.path(DATA_DIR,'GSM7791844_RS03060_filtered_feature_bc_matrix.h5'),
              file.path(DATA_DIR,'GSM7791843_RS03060_fragments.tsv.gz'))
)

read_data_into_seurat <- function(experiment){
    # experiment should be 'gnp', 'pnc', or 'tumor'
    data <- Seurat::Read10X_h5(DATA_LOCATIONS[[experiment]][1])
    print('loading RNA data...')
    obj <- CreateSeuratObject(
        counts=data$'Gene Expression',
        project=experiment,
        assay='RNA',
        min.cells = 10,
        min.features=100
    )
    return(obj)
}

read_annot_helper <- function(df,prefix){
    df <- df %>% 
        filter(str_detect(cell, prefix)) %>%
        mutate(cell = sub(".*#", "", cell)) %>%
        column_to_rownames(., var = "cell")

    return(df)
}

read_cell_annotations <- function(path='data/GSE243609/20241018_scRNA_CellTypeAnno_Table.csv'){
    df = readr::read_csv(path,show_col_types = FALSE)
    annot = list()
    annot$gnp = read_annot_helper(df,"^032103")
    annot$pnc = read_annot_helper(df,"^RS03056")
    annot$tumor = read_annot_helper(df,"^RS03060")
    return(annot)
}

In [ ]:
# Read seurat objects
gnp_data = read_data_into_seurat('gnp')
pnc_data = read_data_into_seurat('pnc')
tumor_data = read_data_into_seurat('tumor')

# format as list of SO
sc_list <- list(
    gnp = gnp_data,
    pnc = pnc_data,
    tumor = tumor_data
)
sc_list

In [ ]:
sc_list[[1]][[]]

In [ ]:
# Annotate with cell ids
annot = read_cell_annotations()
for (sample in names(sc_list)){
    sc_list[[sample]]$annotation <- annot[[sample]]
}

In [ ]:
## QC
qc <- function(sc_list){
    # TODO: make plots pretty colors
    # report number of low-quality cells dropped
    plots = list()
    for (name in names(sc_list)){
        x <- sc_list[[name]]

        # define useful qc metrics
        x[["percent.mt"]] <- PercentageFeatureSet(x, pattern = "^mt-")
        x[["log_count_rna"]] <- log(x$nCount_RNA)
        x[["log_features_rna"]] <- log(x$nFeature_RNA)

        # set thresholds
        ncount_lower_threshold = exp(mean(x$log_count_rna) - 2*sd(x$log_count_rna))
        ncount_upper_threshold = exp(mean(x$log_count_rna) + 2*sd(x$log_count_rna))
        nfeature_lower_threshold = exp(mean(x$log_features_rna) - 2*sd(x$log_features_rna))
        nfeature_upper_threshold = exp(mean(x$log_features_rna) + 2*sd(x$log_features_rna))
        mitochondrial_threshold = mean(x$percent.mt) + 2*sd(x$percent.mt)

        # plot thresholds
        plots[[paste(name,"counts",sep="_")]] <- VlnPlot(x, features="nCount_RNA", log=TRUE, layer="counts") + 
            geom_hline(yintercept=ncount_lower_threshold, color='red') + 
            geom_hline(yintercept=ncount_upper_threshold, color='red')
        plots[[paste(name,"features",sep="_")]] <- VlnPlot(x, features="nFeature_RNA", log=TRUE, layer="counts") +
            geom_hline(yintercept=nfeature_lower_threshold,color='red') + 
            geom_hline(yintercept=nfeature_upper_threshold,color='red')
        plots[[paste(name,"mitochondria",sep="_")]] <- VlnPlot(x, features="percent.mt", log=TRUE, layer="counts") + 
            geom_hline(yintercept=mitochondrial_threshold,color='red')

        # apply thresholds
        sc_list[[name]] <- x %>% 
            subset(nFeature_RNA > nfeature_lower_threshold & nFeature_RNA < nfeature_upper_threshold & 
                   nCount_RNA > ncount_lower_threshold & nCount_RNA < ncount_upper_threshold & 
                   percent.mt < mitochondrial_threshold
                  )
    }
    options(repr.plot.width = 12, repr.plot.height = 12)
    combined_plot <- wrap_plots(plots) + plot_layout(ncol = 3)  # Adjust ncol as needed
    print(combined_plot)
    return(sc_list)
}
sc_list <- qc(sc_list)

In [ ]:
standard_seurat_analysis <- function(seuratobj){
    return (seuratobj %>%
        SCTransform(verbose = FALSE, vst.flavor="v2") %>%
        RunPCA(verbose = FALSE) %>%
        RunUMAP(dims = 1:30, verbose = FALSE) %>%
        FindNeighbors(dims = 1:30, verbose = FALSE) %>%
        FindClusters(verbose = FALSE)
    )
}
regress_mito_seurat_analysis <- function(seuratobj){
    return (seuratobj %>%
        SCTransform(verbose = FALSE, vst.flavor="v2",vars.to.regress = "percent.mt", new.assay.name = "m_SCT") %>%
        RunPCA(verbose = FALSE, assay="m_SCT", reduction.name="m_pca") %>%
        RunUMAP(dims = 1:30, verbose = FALSE, reduction = "m_pca", reduction.name = "m_umap") %>%
        FindNeighbors(dims = 1:30, verbose = FALSE, reduction = "m_pca", graph.name = "m_snn") %>%
        FindClusters(verbose = FALSE, graph.name="m_snn", cluster.name="mito_regressed_cluster")
    )
}

In [ ]:
sc_list <- lapply(X=sc_list, FUN=standard_seurat_analysis)

In [ ]:
sc_list <- lapply(X=sc_list, FUN=regress_mito_seurat_analysis)

In [ ]:
sc_list[[2]][[]]

In [ ]:
dimplots <- function(sc_list){
    plots = list()
        for (name in names(sc_list)){
            x <- sc_list[[name]]
            Idents(object = x) <- "annotation"
            for (reduction in c("umap","m_umap")){
                if (reduction == "umap"){
                    negation = "not"
                } else {
                    negation = ""
                }
                title <- paste0("Clustering of sample ",name,'\n',negation," controlling for mitochondrial read fraction")
                plots[[title]] <- DimPlot(x, reduction=reduction, label=TRUE, repel=TRUE) + 
                    ggtitle(title) + 
                    theme(legend.position = "bottom")
            }
            if (!F){ # don't generate this plot because it's uninformative but keep the code in case it's useful
            frequency_table <- x[[]] %>%
                group_by(seurat_clusters, annotation, mito_regressed_cluster) %>%
                summarise(count = n(), .groups = 'drop')
            plots[[paste0(name," alluvial")]] <- ggplot(frequency_table,
                    aes(axis1 = seurat_clusters,
                        axis2 = annotation,
                        axis3 = mito_regressed_cluster,
                        y = count)) +
                geom_flow(aes(fill = annotation),decreasing=FALSE) +
                geom_stratum(decreasing=FALSE) +
                geom_text(stat = "stratum", decreasing=FALSE,
                    aes(label = after_stat(stratum)))
            }
        }
    options(repr.plot.width = 20, repr.plot.height = 16)
    combined_plot <- wrap_plots(plots) + plot_layout(ncol = 3)
    print(combined_plot)
    return()
}
dimplots(sc_list)

In [ ]:
sc_list <- lapply(X=sc_list, FUN=function(x){
    DefaultAssay(x) <- "m_SCT"
    return(x)
})

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 20)
ncol=5
features=c("Gli1","Gli2","Mki67","Ctnnb1","Birc5","Ntrk3",
           "Trp53","Pten","Tll1","Tex14",
           "Rbfox3","Cntn2","Neurod1","Cacna1e","Kcnk1","Grin2b","Cntnap4","Samd12","Zmat4",
           "Prom1","Myc","Scrt2")
FeaturePlot(sc_list[[1]], slot='data', features=features,ncol=ncol, label=TRUE)
FeaturePlot(sc_list[[2]], slot='data', features=features,ncol=ncol, label=TRUE)
FeaturePlot(sc_list[[3]], slot='data', features=features,ncol=ncol, label=TRUE)

In [ ]:
saveRDS(sc_list, "out/seuratobject_list.rds")

# pseudotime

In [ ]:
sc_list <- readRDS("out/seuratobject_list.rds")

In [ ]:
# check for cells which change cluster identity
test <- sc_list[[2]]
# but regressing out mitochondrial fraction seems to reduce intra-cluster variation

In [ ]:
get_umap_centroids <- function(seuratobj){
    # Get UMAP embeddings
    umap_embeddings <- Embeddings(seuratobj, reduction = 'm_umap')
    
    # Step 2: Get cluster identities
    clusters <- Idents(seuratobj)
    
    # Step 3: Combine embeddings and cluster identities into a data frame
    umap_data <- as.data.frame(umap_embeddings)
    umap_data$cluster <- clusters
    
    # Step 4: Calculate centroids for each cluster
    centroids <- umap_data %>%
      group_by(cluster) %>%
      summarize(across(everything(), mean))
    
    # View centroids
    return(centroids)
}

plot_slingshot_mst <- function(so,subset_so,psobj){
    # helper function plots minimum spanning tree from slingshot analysis onto the umap DimPlot.
    # subset_so: subsetted seurat object only containing clusters of interest
    # psobj: PseudotimeOrdering from slingshot.
    centroids <- get_umap_centroids(subset_so)
    mst <- slingMST(psobj) %>% (igraph::as_edgelist) %>% as.data.frame
    edges <- mst %>%
        left_join(centroids, by = c("V1" = "cluster")) %>%
        rename("mumap_1"="x1","mumap_2"="y1") %>%
        left_join(centroids, by = c("V2" = "cluster")) %>%
        rename("mumap_1"="x2","mumap_2"="y2")
    plt <- DimPlot(so, reduction="m_umap", label=TRUE, label.size=5, repel=TRUE) + 
        geom_point(aes(y=mumap_2,x=mumap_1),data=centroids) + 
        geom_segment(data=edges, aes(x=x1, y=y1, xend=x2, yend=y2)) + 
        ggtitle("Pseudotime trajectory in UMAP space") + 
        theme(plot.title = element_text(hjust = 0.5))
    return(plt)
}

plot_pseudotime_ordering <- function(so,cell_ids="mito_regressed_cluster",pseudotime="avg_pseudotime"){
    # so: seuratobject. Assumes cell_ids and pseudotime columns present in the metadata
    plot <- ggplot(so[[]] %>% na.omit, aes(x = !!sym(pseudotime), y = !!sym(cell_ids), 
                              colour = !!sym(cell_ids))) +
        geom_quasirandom(groupOnX = FALSE) + 
        theme_classic() +
        xlab("Slingshot pseudotime") + ylab("cell type") +
        ggtitle("Cells ordered by Slingshot pseudotime") + 
        theme(plot.title = element_text(hjust = 0.5))
    return(plot)
}

pseudotime_analysis <- function(seuratobj,clusters_to_drop=NULL,start.clus=NULL,end.clus=NULL){
    # clusters_to_drop: clusters to remove from pseudotime analysis, presumably because they come from an independent lineage. Look for outlier clusters in the DimPlot.
    Idents(seuratobj) <- 'mito_regressed_cluster' # use these cluster identities

    subset_so <- seuratobj %>% subset(idents=clusters_to_drop, invert=TRUE) 
    # get cell IDs for clusters of interest
    clustering <- Idents(subset_so)
    # get first 30 PCs for all cells in clusters of interest
    data <- Embeddings(subset_so,reduction='m_pca')[names(clustering), 1:30]
    # run slingshot
    psobj <- slingshot(data = data, clusterLabels = clustering,start.clus=start.clus,end.clus=end.clus)

    plots = list()
    
    # Plot minimum spanning tree
    plots[["mst"]] <- plot_slingshot_mst(seuratobj,subset_so,psobj)

    # Add pseudotime to metadata
    seuratobj$avg_pseudotime <- psobj %>% slingAvgPseudotime
    
    # Plot cells along pseudotime trajectory
    plots[["cell_pseudotime"]] <- plot_pseudotime_ordering(seuratobj)

    # Plot Neurod1 expression against pseudotime
    DefaultAssay(seuratobj) <- "m_SCT"
    plots[["Neurod1_pseudotime"]] <- FeatureScatter(seuratobj,feature1='avg_pseudotime',feature2='Neurod1',pt.size=0.5,slot='scale.data') + 
        geom_smooth(na.rm=TRUE,method='gam',formula = y ~ s(x, bs = "gp")) + 
        ggtitle("Neurod1 expression over pseudotime") +
        theme(plot.title = element_text(hjust = 0.5))

    options(repr.plot.width = 18, repr.plot.height = 6)
    combined_plot <- wrap_plots(plots) + plot_layout(ncol = 3)  # Adjust ncol as needed
    print(combined_plot)
    
    return(list(seuratobject=seuratobj,pseudotime=psobj,plots=plots))
}

In [ ]:
gnp_result <- pseudotime_analysis(sc_list[[1]],clusters_to_drop=c(7,8,9,11,12,13,14,15),end.clus=5)

In [ ]:
pnc_result <- pseudotime_analysis(sc_list[[2]],clusters_to_drop=c(7,9),start.clus=5,end.clus=2)

In [ ]:
tumor_result <- pseudotime_analysis(sc_list[[3]],clusters_to_drop=c(9,10),start.clus=0,end.clus=4)

In [ ]:
tumor_result[[1]][[]]

In [ ]:
pseudotime_heatmap <- function(seuratobj,features){
    DefaultAssay(seuratobj) <- "m_SCT"
    # order cells by pseudotime
    x<- FetchData(seuratobj, vars = "avg_pseudotime") %>% as.data.frame
    order.index <- order(x$avg_pseudotime, decreasing = FALSE)
    cell.order <- rownames(x)[order.index]
    
    # hack hack add pseudotime to heatmap by making a new assay with avg-pseudotime as a gene
    seuratobj[['heatmap']] <- CreateAssayObject(data = t(x = FetchData(object = seuratobj, layer = 'scale.data', vars = c(features,'avg_pseudotime'))))
    seuratobj[["heatmap"]]$data['avg-pseudotime', ] <- seuratobj[["heatmap"]]$data['avg-pseudotime', ]/20
    
    # plot
    DoHeatmap(object = seuratobj, features = c(features,'avg-pseudotime'), assay = 'heatmap', slot = 'data',group.by="orig.ident", cells=cell.order)
}

features=c("Gli1","Gli2","Mki67","Ctnnb1","Birc5","Ntrk3",
           "Trp53","Pten","Tll1","Tex14",
           "Rbfox3","Cntn2","Neurod1","Cacna1e","Kcnk1","Grin2b","Cntnap4","Samd12","Zmat4",
           "Prom1","Myc","Scrt2")

options(warn=-1)
pseudotime_heatmap(gnp_result[[1]],features)
pseudotime_heatmap(pnc_result[[1]],features)
pseudotime_heatmap(tumor_result[[1]],features)
options(warn=0)

In [ ]:
library(RColorBrewer)

pseudotime_curves <- function(seuratobj,features){
    num_features <- length(features)
    # Generate a color palette (up to 12 colors with 'Set3')
    colors <- if (num_features > 12) {
      colorRampPalette(brewer.pal(12, "Set3"))(num_features)
    } else {
      brewer.pal(num_features, "Set3")
    }


    # Create a mapping of colours to feature names
    colour_to_feature <- data.frame(colour = colors, 
                                 feature = features)
    
    DefaultAssay(seuratobj) <- "m_SCT"
    data <- FetchData(object = seuratobj, vars = c("avg_pseudotime", features), layer = "data") %>%
        pivot_longer(cols = features, names_to = 'gene', values_to = 'expression')
    
    options(repr.plot.width = 18, repr.plot.height = 6)
    p <- ggplot(data, aes(x=avg_pseudotime,y=expression, color=gene)) +
        geom_smooth(se=FALSE,na.rm=TRUE,method='gam',formula = y ~ s(x, bs = "gp")) +
        #scale_color_manual(values = setNames(colour_to_feature$colour, colour_to_feature$feature)) +  # Set colors
        scale_color_manual(values = setNames(colors, features)) +  # Set colors directly
        labs(color = "Feature") + 
        theme_classic()

    # Build the ggplot object to extract smoothed data
    p_built <- ggplot_build(p)
    
    # Extract the smoothed data for each feature
    endpoints <- p_built$data[[1]] %>%
        mutate(xc=x,yc=y) %>% # rename because variable namespace too crowded for which.max
        group_by(colour) %>%
        reframe(
            x = max(xc),
            y = yc[base::which.max(xc)],
            #feature = colour_to_feature$feature[match(colour, setNames(colour_to_feature$colour, colour_to_feature$feature))],
            feature = features[match(colour, colors)],  # Match directly using colors
        ) %>% unique
    # Add labels for each feature at the endpoints of the curves
    p <- p + geom_label_repel(data = endpoints, aes(x = x, y = y, label = feature), 
                       #vjust = -0.5, hjust = 1.5, 
                       size = 4, color = "black")

    startpoints <- p_built$data[[1]] %>%
        mutate(xc=x,yc=y) %>% # rename because variable namespace too crowded for which.max
        group_by(colour) %>%
        reframe(
            x = min(xc),
            y = yc[base::which.min(xc)],
            #feature = colour_to_feature$feature[match(colour, setNames(colour_to_feature$colour, colour_to_feature$feature))],
            feature = features[match(colour, colors)],  # Match directly using colors
        ) %>% unique
    # Add labels for each feature at the endpoints of the curves
    p <- p + geom_label_repel(data = startpoints, aes(x = x, y = y, label = feature), 
                       #vjust = -0.5, hjust = 1.5, 
                       size = 4, color = "black")
    print(p)
    return(list(endpoints=endpoints,p_built=p_built,color_to_feature=colour_to_feature))
}

subfeatures = c("Gli1","Gli2","Mki67","Neurod1","Birc5","Ntrk3",
           "Rbfox3","Cntn2","Ctnnb1","Cacna1e","Grin2b","Samd12")

options(warn=-1)
out = pseudotime_curves(gnp_result[[1]],subfeatures)
out = pseudotime_curves(pnc_result[[1]],subfeatures)
out = pseudotime_curves(tumor_result[[1]],subfeatures)
options(warn=0)

In [ ]:
options(warn=0)

In [ ]:
saveRDS(psobj, "out/slingshot_pseudotimeordering.rds")


# TODO
- Label cells by cell type annotation
- Set start and end points
- Plot gene expression of Neurod1 etc. 

# Pseudotime illustration

In [ ]:
?ggsave

In [ ]:
plots = list()

seuratobj = sc_list[[2]]
Idents(object = seuratobj) <- "annotation"

tmp <- FeaturePlot(seuratobj, slot='data', features = c("Mki67","Rbfox3"), blend = TRUE, combine=FALSE)
plots[[1]] <- tmp[[3]] + scale_color_manual(labels=c('Ki67-/Neun-','Ki67+/Neun-','Ki67-/Neun+','Ki67+/Neun+'),
                                           values=c("grey","red","green","yellow")) + 
    ggtitle("Ki67/NeuN expression")

plots[[2]] = DimPlot(seuratobj, reduction="m_umap", label=TRUE, repel=TRUE, label.size=5) + 
    ggtitle("Cell annotations") + 
    theme(plot.title = element_text(hjust = 0.5))

seuratobj = pnc_result[[1]]
subset_so <- seuratobj %>% subset(idents=c(7,9), invert=TRUE)
psobj = pnc_result[[2]]
Idents(object = seuratobj) <- "mito_regressed_cluster"
plots[[3]] = plot_slingshot_mst(seuratobj,subset_so,psobj) + theme(legend.position = "none") +
    ggtitle("Pseudotime trajectory")

options(repr.plot.width = 20, repr.plot.height = 8)
combined_plot <- wrap_plots(plots) + plot_layout(ncol = length(plots)) + 
    plot_annotation(title="Mouse Ptch1- preneoplastic cells from\nShiraishi et al. 2023",
                   theme = theme(plot.title = element_text(hjust = 0.5, size=16, face='bold')))
print(combined_plot)
combined_plot <- combined_plot + plot_layout(ncol = 1)
ggsave("out/pseudotime_example.png",width=6,height=12)

# Atoh1 in pnc

In [ ]:
sample = sc_list[[2]]
features = c('Atoh1','Neurod1')
FeaturePlot(sample, slot='data', features=features,ncol=2, label=TRUE)

# Dead code

In [ ]:
# Signac hangs on CreateChromatinAssay during "computing hash" step.
# possible causes:
#  Apple silicon architecture?
#  Just requires a ton of resources?
#  May need to load multiome data on shirokane, or use a different tool.

library(Signac)
library(BSgenome.Mmusculus.UCSC.mm10)
library(EnsDb.Mmusculus.v79)

ANNOTATION <- GetGRangesFromEnsDb(ensdb = EnsDb.Mmusculus.v79)
seqlevelsStyle(ANNOTATION) <- 'UCSC'
genome(ANNOTATION) <- "mm10"

print('loading ATAC data...')
atac_counts <- data$Peaks
grange.counts <- StringToGRanges(rownames(atac_counts), sep = c(":", "-"))
grange.use <- seqnames(grange.counts) %in% standardChromosomes(grange.counts)
atac_counts <- atac_counts[as.vector(grange.use), ]
obj[["ATAC"]] <- CreateChromatinAssay(
    counts = atac_counts,
    sep = c(":", "-"),
    genome = 'mm10',
    fragments = DATA_LOCATIONS[[experiment]][2],
    min.cells = 10,
    min.features = 100,
    annotation = ANNOTATION
)